# Simulating lightcurves of lensed SNe Ia

This example requires the development version of simurvey found in this feature branch:
https://github.com/ufeindt/simsurvey/tree/feature/manipulate-transient-parameters

The changes there allow easy manipulation of the transient parameters, e.g. to change the amplitude based on lens magnification or duplicate a transient and shift its RA, Dec and $t_0$ slightly to simulate multiple images.

In [1]:
import os
home_dir = os.environ.get('HOME')

# Please enter the path to where you have placed the Schlegel, Finkbeiner & Davis (1998) dust map files
# You can also set the environment variable SFD_DIR to this path (in that case the variable below should be None)
sfd98_dir = os.path.join(home_dir, 'data/sfd98')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from astropy.cosmology import Planck15
import simsurvey
import sncosmo
import simsurvey_tools as sst

In [3]:
simsurvey.__version__

'0.7.0.dev3'

In [4]:
simsurvey.__path__

['/home/ufeindt/anaconda3/envs/simsurvey-dev/lib/python3.7/site-packages/simsurvey-0.7.0.dev3-py3.7.egg/simsurvey']

In [5]:
fields = sst.load_ztf_fields()

In [6]:
fields['ra'][682], fields['dec'][682]

(274.7095, 33.35)

In [7]:
# Load the CCD corners from file
ccd_corners = np.genfromtxt('data/ztf_quadrants.tbl', skip_header=1)
ccds = [ccd_corners[np.array([0,1,2,3])+4*k, :2] for k in range(64)]

In [8]:
bands = { 
  'ztfr' : 'data/ztfr_eff.txt',
  'ztfg' : 'data/ztfg_eff.txt',
  'ztfi' : 'data/ztfi_eff.txt',
  }

for bandname in bands.keys() :
    fname = bands[bandname]
    b = np.loadtxt(fname)
    band = sncosmo.Bandpass(b[:,0], b[:,1], name=bandname)
    sncosmo.registry.register(band, force=True)

In [9]:
obs = {'time': [], 'field': [], 'band': [], 'skynoise': [], 'comment': [], 'zp': []}

mjd_start = 58239.5
for k in range(0, 61, 3):
    obs['time'].extend([mjd_start + k + l/24. for l in range(2)])
    obs['field'].extend([683 for l in range(2)])
    obs['band'].extend(['ztfg', 'ztfr'])
    obs['skynoise'].extend([1262 for l in range(2)])
    obs['zp'].extend([30 for l in range(2)])
    obs['comment'].extend(['MSIP', 'MSIP'])
    
#for k in range(61):
#    obs['time'].extend([mjd_start + k + (l+2)/24. for l in range(3)])
#    obs['field'].extend([686 for l in range(3)])
#    obs['band'].extend(['ztfg', 'ztfg', 'ztfg'])
#    obs['skynoise'].extend([1262 for l in range(3)])
#    obs['zp'].extend([30 for l in range(3)])
#    obs['comment'].extend(['fast-g', 'fast-g', 'fast-g'])    
     
plan = simsurvey.SurveyPlan(time=obs['time'], band=obs['band'], obs_field=obs['field'],
                            skynoise=obs['skynoise'], comment=obs['comment'],
                            zp=obs['zp'],
                            fields={k: v for k, v in fields.items()
                                    if k in ['ra', 'dec', 'field_id',
                                            'width', 'height']},
                            ccds=ccds)

In [10]:
plan.pointings

time,band,zp,skynoise,RA,Dec,field,ccd,comment
float64,str4,int64,int64,float64,float64,int64,float64,str4
58239.5,ztfg,30,1262,274.7095,33.35,683,nan,MSIP
58239.541666666664,ztfr,30,1262,274.7095,33.35,683,nan,MSIP
58242.5,ztfg,30,1262,274.7095,33.35,683,nan,MSIP
58242.541666666664,ztfr,30,1262,274.7095,33.35,683,nan,MSIP
58245.5,ztfg,30,1262,274.7095,33.35,683,nan,MSIP
58245.541666666664,ztfr,30,1262,274.7095,33.35,683,nan,MSIP
58248.5,ztfg,30,1262,274.7095,33.35,683,nan,MSIP
58248.541666666664,ztfr,30,1262,274.7095,33.35,683,nan,MSIP
58251.5,ztfg,30,1262,274.7095,33.35,683,nan,MSIP


In [11]:
mjd_range = (plan.pointings['time'].min(), plan.pointings['time'].max())
ra_range = (270., 280.)
dec_range = (29., 37.5)

In [12]:
z_max = 0.1

tr = simsurvey.get_transient_generator([0., z_max],
                                       transient='Ia', 
                                       template='salt2',
                                       ra_range=ra_range,
                                       dec_range=dec_range,
                                       mjd_range=[mjd_range[0],
                                                  mjd_range[1]],
                                       sfd98_dir=sfd98_dir,
                                       ntransient=5)

## New functions
### get_lc_param()
This function allows quick retrieval of lightcurve parameters.

In [13]:
print("Redshifts of all SNe (for reference):", tr.zcmb)
print()
print("Retrieve 3rd SNe - tr.get_lc_param(2)")
param = tr.get_lc_param(2)
print(param)

Redshifts of all SNe (for reference): [0.09803075 0.09349302 0.08174144 0.09793891 0.07799387]

Retrieve 3rd SNe - tr.get_lc_param(2)
{'z': 0.08174143625170985, 't0': 58269.91281342771, 'ra': 275.6445136065594, 'dec': 34.823454565169556, 'x0': 0.0005126631106296095, 'x1': -0.6454243486941382, 'c': 0.0069765190984302314, 'mwebv': 0.018623711078597713, 'mwebv_sfd98': 0.04544335864877227}


### remove_lc_param()
This function removes a transient from the generator.

In [14]:
print("Redshifts of all SNe (before):", tr.zcmb)
tr.remove_lc_param(2)
print("Redshifts of all SNe (after removing the 3rd SN):", tr.zcmb)

Redshifts of all SNe (before): [0.09803075 0.09349302 0.08174144 0.09793891 0.07799387]
Redshifts of all SNe (after removing the 3rd SN): [0.09803075 0.09349302 0.09793891 0.07799387]


### add_lc_param()
This function adds a new SN. The easiest way to generate the correct input is to get a dictionary of parameters and change it.

In [15]:
print("Redshifts of all SNe (after removing the 3rd SN):", tr.zcmb)
tr.add_lc_param(**param)
print("Redshifts of all SNe (after re-adding the previously 3rd SN):", tr.zcmb)

Redshifts of all SNe (after removing the 3rd SN): [0.09803075 0.09349302 0.09793891 0.07799387]
Redshifts of all SNe (after re-adding the previously 3rd SN): [0.09803075 0.09349302 0.09793891 0.07799387 0.08174144]


### pop_lc_param()
This function combines `get_lc_param()` and `remove_lc_param()` into one step.

In [16]:
print("Redshifts of all SNe (before):", tr.zcmb)
print()
print("Pop 3rd SNe - tr.pop_lc_param(2)")
param = tr.pop_lc_param(2)
print(param)
print()
print("Redshifts of all SNe (after popping the 3rd SN):", tr.zcmb)

Redshifts of all SNe (before): [0.09803075 0.09349302 0.09793891 0.07799387 0.08174144]

Pop 3rd SNe - tr.pop_lc_param(2)
{'z': 0.09793890840555515, 't0': 58254.23935834847, 'ra': 271.1850615336316, 'dec': 33.5132911294908, 'x0': 0.00041149597275357897, 'x1': -0.137766462239143, 'c': 0.002391247715833312, 'mwebv': 0.06205026114166769, 'mwebv_sfd98': 0.04530093592073473}

Redshifts of all SNe (after popping the 3rd SN): [0.09803075 0.09349302 0.07799387 0.08174144]


### set_lc_param()
If you just wish to change a few parameters of a transient, use this function.

In [17]:
print('3rd SN:')
param = tr.get_lc_param(2)
print(param)
print()
tr.set_lc_param(2, t0=param['t0']+10, x0=param['x0']*10)
print('3rd SN with magnification and time delay:')
param = tr.get_lc_param(2)
print(param)

3rd SN:
{'z': 0.07799386809285375, 't0': 58285.12585648938, 'ra': 274.39420110937675, 'dec': 30.086883121673463, 'x0': 0.0006948633856780484, 'x1': 0.7498817696505224, 'c': -0.035979162162463815, 'mwebv': 0.09570515950469959, 'mwebv_sfd98': 0.10142050542753037}

3rd SN with magnification and time delay:
{'z': 0.07799386809285375, 't0': 58295.12585648938, 'ra': 274.39420110937675, 'dec': 30.086883121673463, 'x0': 0.006948633856780484, 'x1': 0.7498817696505224, 'c': -0.035979162162463815, 'mwebv': 0.09570515950469959, 'mwebv_sfd98': 0.10142050542753037}


## The simulation should hopefully still run properly

In [18]:
survey = simsurvey.SimulSurvey(generator=tr, plan=plan)

lcs = survey.get_lightcurves(
    progress_bar=True, notebook=True # If you get an error because of the progress_bar, delete this line.
)
print(tr.ntransient, len(lcs.lcs))

Determining field IDs for all objects


FloatProgress(value=0.0)


Generating lightcurves


FloatProgress(value=0.0)


4 3
